# **Transfer learning:** pre-trained text embeddings

For this example we will use a pre-trained text embedding model from TensorFlow Hub called **google/nnlm-en-dim128/2**.
One way to represent the text is to convert sentences into embeddings vectors. We can use a pre-trained text embedding as the first layer, which will have three advantages:

*   We don't have to worry about text preprocessing.
*   We can benefit from transfer learning.
*   The embedding has a fixed size, so it's simpler to process.

Let's first create a Keras layer that uses a TensorFlow Hub model to embed the sentences.

In [53]:
# Libraries
import numpy as np
import tensorflow_datasets as tfds
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers 
import pandas as pd

## Loading IMBD Dataset


We’ll work with the IMDB dataset: a set of 50,000 highly polarized reviews from the Internet Movie Database. They’re split into 25,000 reviews for training and 25,000 reviews for testing, each set consisting of 50% negative and 50% positive reviews. The parameter num_words controls how many words different we want to use.


We are going to download the dataset using [TFDS](https://www.tensorflow.org/datasets). TFDS provides a collection of ready-to-use datasets for use with TensorFlow, Jax, and other Machine Learning frameworks.


In [54]:
!pip install -q tensorflow_datasets

In [55]:
dataset, info = tfds.load('imdb_reviews', with_info=True,
                          as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

train_dataset.element_spec

(TensorSpec(shape=(), dtype=tf.string, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None))

Initially this returns a dataset of (text, label pairs):

In [56]:
for example, label in train_dataset.take(1):
    print('text: ', example.numpy())
    print('--'*100)
    print('label: ', label.numpy())

text:  b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
label:  0


In [57]:
BUFFER_SIZE = 1000
BATCH_SIZE = 128

In [58]:
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [59]:
for example, label in train_dataset.take(1):
    print('texts: ', example.numpy()[:3])
    print()
    print('labels: ', label.numpy()[:3])

texts:  [b'Erich Rohmer\'s "L\'Anglaise et le duc" makes a perfect companion piece to Peter Watkins\' "La Commune (Paris 1871)." Both films -screened at this year\'s Toronto International Film Festival- ironically illustrate how history is shaped to by the tellers of the tale. Ironic, given the tragic events that were taking place in the U.S. during the festival.<br /><br />Set in Paris during the French Revolution, the movie, based on Grace Elliott\'s (Lucy Russell) "Memoirs," is a first-hand account of how she survived those heady but dangerous days. She also details her relationship with The Duke of Orleans (played by Jean-Claude Dreyfus), who, in contrast to herself, is a supporter of the Revolution.<br /><br />True to form, you don\'t know whose side of history Rohmer is going to come down on. One of the earliest of the French "New Wave" filmmakers, Rohmer has often been criticized for being too conservative. After all, in the midst of the rebelling-youth-Viet-Nam days of the late

In [60]:
def show_loss_accuracy_evolution(history):
    
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Sparse Categorical Crossentropy')
    ax1.plot(hist['epoch'], hist['loss'], label='Train Error')
    ax1.plot(hist['epoch'], hist['val_loss'], label = 'Val Error')
    ax1.grid()
    ax1.legend()

    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Accuracy')
    ax2.plot(hist['epoch'], hist['accuracy'], label='Train Accuracy')
    ax2.plot(hist['epoch'], hist['val_accuracy'], label = 'Val Accuracy')
    ax2.grid()
    ax2.legend()

    plt.show()


### Transfer learning: pre-trained text embeddings

For this example we will use a **pre-trained text embedding model** from [TensorFlow Hub](https://tfhub.dev) called [google/nnlm-en-dim128/2](https://tfhub.dev/google/nnlm-en-dim128/2).

[TensorFlow Hub](https://tfhub.dev/) has hundreds of trained, ready-to-deploy machine learning models.  You can find more [text embedding models](https://tfhub.dev/s?module-type=text-embedding) on TFHub.

One way to represent the text is to convert sentences into embeddings vectors. We can use a pre-trained text embedding as the first layer, which will have three advantages:

One way to represent the text is to convert sentences into embeddings vectors. We can use a pre-trained text embedding as the first layer, which will have three advantages:

*   we don't have to worry about text preprocessing,
*   we can benefit from transfer learning,
*   the embedding has a fixed size, so it's simpler to process.

For this example we will use a pre-trained text embedding model from TensorFlow Hub called google/nnlm-en-dim128/2.
Let's first create a Keras layer that uses a TensorFlow Hub model to embed the sentences.

In [61]:
!pip install tensorflow-hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [62]:
import tensorflow_hub as hub

embedding = "https://tfhub.dev/google/nnlm-en-dim128/2"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)

In [63]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(25, activation='sigmoid'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_2 (KerasLayer)  (None, 128)               124642688 
                                                                 
 dense_19 (Dense)            (None, 25)                3225      
                                                                 
 dense_20 (Dense)            (None, 1)                 26        
                                                                 
Total params: 124,645,939
Trainable params: 124,645,939
Non-trainable params: 0
_________________________________________________________________


In [64]:
model.compile(loss='BinaryCrossentropy',
              optimizer='adam',
              metrics=['accuracy'])
history = model.fit(train_dataset, epochs=1,
                    validation_data=test_dataset, 
                    validation_steps=3)

196/196 [==============================] - 17s 84ms/step - loss: 0.4781 - accuracy: 0.7918 - val_loss: 0.3729 - val_accuracy: 0.8438


In [65]:
results = model.evaluate(test_dataset)
print('Test Loss: {}'.format(results[0]))
print('Test Accuracy: {}'.format(results[1]))

196/196 [==============================] - 4s 21ms/step - loss: 0.3232 - accuracy: 0.8664
Test Loss: 0.3232058882713318
Test Accuracy: 0.8664000034332275


In [66]:
reviews = ['the film was really bad and i am very disappointed',
           'The film was very funny entertaining and good we had a great time . brilliant film',
           'this film was just brilliant',
          'This movie has been a disaster',
           'the movie is not bad']
predictions = model.predict(np.array(reviews))

for review, pred in zip(reviews, predictions.flatten()):
    print()
    print(review)
    print('Sentiment: ', np.round(pred, 2))


the film was really bad and i am very disappointed
Sentiment:  0.25

The film was very funny entertaining and good we had a great time . brilliant film
Sentiment:  0.93

this film was just brilliant
Sentiment:  0.39

This movie has been a disaster
Sentiment:  0.26

the movie is not bad
Sentiment:  0.07
